In [2]:
from sys import displayhook
from binance.client import Client
from binance.enums import *
import pandas as pd
import time
from datetime import datetime
from sqlalchemy import create_engine
import numpy as np
from math import log10
from binance import BinanceSocketManager
import logging
import threading
import time
import asyncio

api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)
bsm = BinanceSocketManager(client)

In [3]:
api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)
db_data = 'mysql+mysqldb://' + 'admin' + ':' + '0323A8E3DB' + '@' + 'coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com' + ':3306/' + 'crypto_db' + '?charset=utf8mb4'
engine = create_engine(db_data)

In [4]:
date_recent_checked = "2022-04-25 22:00"
trend_table_rowcount_old = 100

In [5]:
def createframe(msg):
    df = pd.DataFrame([msg]) 
    df = df.loc[:,['s', 'E', 'p']]
    df.columns = ('symbol' , 'Time', 'Price') 
    df.Price = df.Price.astype(float) 
    df.Time = pd.to_datetime(df.Time, unit='ms') 
    return df

In [6]:
async def buysocket(ticker, buy_amt, SL=0.99, Target=1.01, open_position=False):
    print("in socket")
    socket = bsm.trade_socket(ticker)
    order = client.create_order(symbol=ticker, side='BUY', type='MARKET', quantity=buy_amt)
    buyprice = float(order['fills'][0]['price'])
    open_position = True
    while open_position:
        await socket.__aenter__()
        msg = await socket.recv()
        df = createframe(msg)
        print(f'close = {df.Price.values}; target = {buyprice * Target}; stop = {buyprice * SL}', end = "\r")
        # print(f'target = {buyprice * Target}')
        # print(f'stop = {buyprice * SL}')
        if df.Price.values <= buyprice * SL  or df.Price.values >= buyprice * Target:
            order = client.create_order(symbol=ticker, side='BUY', type='MARKET', quantity=buy_amt)
            break

In [7]:

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,datefmt="%H:%M:%S")

def bg_async(func):
    def wrapper():
        loop = asyncio.new_event_loop()
        loop.run_until_complete(func)
    threading.Thread(target=wrapper).start()

def main():
    waiting = True
    check_data = False
    first = True
    while(True):
        if(waiting):
            if datetime.now().minute < 20:
                trend_table_status = pd.read_sql_query("show table status like 'crypto_trend'", engine)
                trend_table_rowcount_new = trend_table_status['Rows'].values[0]
                if (trend_table_rowcount_new > trend_table_rowcount_old):
                    check_data = True
                    waiting = False
                else:
                    #time.sleep(5)
                    await asyncio.sleep(5)
            else:
                #time.sleep(60)
                await asyncio.sleep(60)

        if(check_data):
            logging.info("algo    : pre sleep")
            time.sleep(55)
            logging.info("algo    : post sleep")
            if first:
                #thread = threading.Thread(target=asyncio.run, args=(buysocket('BTCUSDT', 0.0005),))
                
                bg_async(buysocket('BTCUSDT', 0.0005))
                logging.info("Main    : before running thread")
                thread.start()
                logging.info("Main    : after running thread")
                first = False
            check_data = False
            waiting = True

In [8]:
await main()

22:17:01: algo    : pre sleep
22:17:56: algo    : post sleep
22:17:56: Main    : before running thread
22:17:56: Main    : after running thread
22:17:56: algo    : pre sleep


in socket


Exception in thread Thread-3:
Traceback (most recent call last):
  File "C:\Users\ellio\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\ellio\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ellio\anaconda3\lib\asyncio\runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "C:\Users\ellio\anaconda3\lib\asyncio\base_events.py", line 616, in run_until_complete
    return future.result()
  File "C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_19204\2585404920.py", line 9, in buysocket
  File "C:\Users\ellio\anaconda3\lib\site-packages\binance\streams.py", line 197, in recv
    res = await asyncio.wait_for(self._queue.get(), timeout=self.TIMEOUT)
  File "C:\Users\ellio\anaconda3\lib\asyncio\tasks.py", line 494, in wait_for
    return fut.result()
  File "C:\Users\ellio\anaconda3\lib\asyncio\queues.py", line 163, in get
    await getter
RuntimeError: Task 

In [ ]:

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO,
                    datefmt="%H:%M:%S")

logging.info("Main    : before creating thread")
x = threading.Thread(target=asyncio.run(), args=(1,))
logging.info("Main    : before running thread")
x.start()
logging.info("Main    : wait for the thread to finish")
# x.join()
logging.info("Main    : all done")

19:46:54: Main    : before creating thread


TypeError: run() missing 1 required positional argument: 'main'

In [ ]:
async def algo():
    waiting = True
    check_data = False
    first = True
    while(True):
        if(waiting):
            if datetime.now().minute < 20:
                trend_table_status = pd.read_sql_query("show table status like 'crypto_trend'", engine)
                trend_table_rowcount_new = trend_table_status['Rows'].values[0]
                if (trend_table_rowcount_new > trend_table_rowcount_old):
                    check_data = True
                    waiting = False
                else:
                    #time.sleep(5)
                    await asyncio.sleep(5)
            else:
                #time.sleep(60)
                await asyncio.sleep(60)

        if(check_data):
            print('algo bit a')
            asyncio.sleep(5)
            time.sleep(55)
            print('algo bit b')
            if first:
                print("here")
                task2 = asyncio.create_task(buysocket('BTCUSDT', 0.00025))
                first = False
            check_data = False
            waiting = True

In [ ]:
def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

In [ ]:
await buysocket('BTCUSDT', 0.00025)

in socket
